# Profitable Ad-Based App Opportunities for iOS and Google Play Markets

## About This Project

In this sample project, we are acting as Data Analysts for mock company Evergreen Apps, LLC. Evergreen Apps builds free Android and iOS apps. Their primary source of revenue is via in-app ads. Therefore, **we want to build free apps which will attract the most users** with frequent views to produce ad revenue. We will explore questions such as:
- What kinds of free apps are most commonly built? 
- What kinds of free apps have the most users?

#### **Contents**
- [Opening The Data](#opening)
- [Exploring The Data](#exploring)
- [Cleaning The Data](#cleaning)
- [Analyzing The Data](#analyzing)
    - [Most Common Genres](#common)
    - [Most Popular Genres](#popular)
- [Conclusion](#conclusion)
- [Appendix](#appendix)

## Opening The Data <a id="opening"></a>

To determine what kinds of apps will be most lucrative for Evergreen Apps, LLC to build, we will explore data regarding apps on the Apple iOS App Store and Google Play App Store. There are millions of available apps, so collecting the data ourselves is cost prohibitive. Instead, we will be using two free datasets available on Kaggle which provide a sample of app data from each store:

- **[iOS Dataset on Kaggle](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps)**: Contains data on approximately 7,000 iOS apps from the Apple App Store. You can download it directly [here](https://dq-content.s3.amazonaws.com/350/AppleStore.csv).
- **[Android Dataset on Kaggle](https://www.kaggle.com/lava18/google-play-store-apps)**: Contains data on approximately 10,000 Android apps from Google Play. You can download it directly [here](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv).

To begin, let's first open the datasets:

In [1]:
from csv import reader

### The iOS App Store Dataset ###
opened_file = open('AppleStore.csv')
read_file = reader(opened_file)
ios = list(read_file)
ios_header = ios[0]
ios = ios[1:]

### The Google Play Store Dataset ###
opened_file = open('googleplaystore.csv')
read_file = reader(opened_file)
android = list(read_file)
android_header = android[0]
android = android[1:]

## Exploring The Data <a id="exploring"></a>
We will need a way to easily explore the data as we go along. Let's write a function, `explore_data()`, which will allow us to slice the dataset and print each row of data on a new line. We can also use it to show the total number of rows and columns for any dataset.

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n')

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

Using our `explore_data()` function, let's examine a couple rows of each dataset. 

In [3]:
print('Sample of iOS Dataset: \n---------------------------')
print(ios_header)
print('\n')
explore_data(ios, 0, 1, True)

print('\n')

print('Sample of Android Dataset: \n---------------------------')
print(android_header)
print('\n')
explore_data(android, 0, 1, True)

Sample of iOS Dataset: 
---------------------------
['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


Number of rows: 7197
Number of columns: 16


Sample of Android Dataset: 
---------------------------
['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


We can see the iOS dataset contains 7197 apps, and the Android dataset contains 10841 apps. At first glance, some useful data that each dataset shares includes the app name, genre, and rating. 

We're also very interested in the total number of users using each app. For the iOS dataset, this is best represented by the rating count total. For the Android dataset, we can use the total number of installs. 

## Cleaning The Data <a id="cleaning"></a>

Evergreen Apps, LLC will only be building free apps for English speaking audiences, so in our analysis, we only want to consider apps that are similar to those that would be built by our company. Therefore, we must first clean the data by removing any apps that don't fit our criteria. 

We'll remove:
1. Entries with invalid or incomplete data.
2. Duplicate apps.
3. Non-English apps.
4. Paid apps.

### 1) Deleting Bad Data <a id="bad-data"></a>

[A Kaggle user has identified](https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015) that row 10472 of our Android dataset has an empty `Content Rating`.

In [4]:
print(android_header, '\n')
print(android[10472])

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


Let's delete that row, `android[10472]`, from our dataset.

In [5]:
del android[10472]

print(android[10472])

['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']


The row has been successfully deleted!

### 2) Removing Duplicate Entries <a id="dupes"></a>

#### Part 1: Counting the Duplicates

We don't want any duplicate apps in our datasets which could skew the data. To begin, let's count how many duplicates there are:

In [6]:
## Counting Android Duplicates ##
android_duplicate_apps = []
android_unique_apps = []

for app in android:
    if app[0] in android_unique_apps:
        android_duplicate_apps.append(app[0])
    else:
        android_unique_apps.append(app[0])
        
print('Number of duplicate Android apps: ', len(android_duplicate_apps))
print('Examples of duplicate Android apps: ', android_duplicate_apps[:5])

Number of duplicate Android apps:  1181
Examples of duplicate Android apps:  ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings']


In [7]:
## Counting iOS Duplicates ##
ios_duplicate_apps = []
ios_unique_apps = []

for app in ios:
    if app[0] in ios_unique_apps:
        ios_duplicate_apps.append(app[0])
    else:
        ios_unique_apps.append(app[0])
        
print('Number of duplicate iOS apps: ', len(ios_duplicate_apps))
print('Examples of duplicate iOS apps: ', ios_duplicate_apps[:15])

Number of duplicate iOS apps:  0
Examples of duplicate iOS apps:  []


There are a lot of duplicate Android apps to delete (1181 of them!), and no iOS duplicates. Let's work on removing the Android duplicates.

#### Part 2: Removing the Duplicates


We could delete the duplicates at random, but for the purposes of our analysis, there is a better way. Since we will be analyzing which apps attract the most users, we'll assume the entry with the most reviews is the most current, and we'll delete the older entries.

First, let's extract the entries with the most ratings for each app, and store it in a dictionary called `reviews_max`. We can double check we got them all by verifying the number of values in `reviews_max` is equal to the number of values in the `android` dataset minus the number of duplicates we identified above.

In [8]:
## Identify and Store Android Entries with the Most Reviews ##
reviews_max = {}

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    # Store the entry with the most reviews
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    
    # Store new entries
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

print('Expected Number of Apps: ' + str(len(android) - len(android_duplicate_apps)))
print('Total Apps: ' + str(len(reviews_max)))

Expected Number of Apps: 9659
Total Apps: 9659


Next, we'll separate out the duplicates. For each app in the `android` dataset, we'll compare its number of ratings to the number of ratings in `reviews_max`. If they are equal, we know that is the entry with the most views. We'll keep that entry by appending it to a new list called `android_clean`. Entries that have a lower number of ratings will simply be ignored.

_But wait!_ There's one possible edge case we haven't accounted for: What if there are multiple entries for the same app that also have the same number of reviews? 

To account for this, as we add new entries to the `android_clean` dataset, we'll also  store the name of each app into a separate list called `already_added`. If the app has already been added to `android_clean`, we can simply skip it. So, in our `if` statement we will check that the app is `not in already_added`. This will prevent us from adding any duplicates that also happen to have the same number of reviews.

In [9]:
## Separate Unique Apps into a Clean List ##
android_clean = []
already_added = []

for app in android:
    name = app[0]
    n_reviews = float(app[3])
    
    if n_reviews == reviews_max[name] and name not in already_added:
        android_clean.append(app)
        already_added.append(name)

Let's quickly explore the new clean dataset and verify that we have the expected number of unique apps, 9659. 

In [10]:
explore_data(android_clean, 0, 3, True)

['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9659
Number of columns: 13


Success!

### 3) Removing Non-English Apps <a id="non-english"></a>

There are apps in both the iOS and Android datasets that do not have English names.

In [11]:
print(ios[813][1])
print(android_clean[4412][0])

爱奇艺PPS -《欢乐颂2》电视剧热播
中国語 AQリスニング


Since our company will only be building English apps, we want to remove these. One way to do this is by removing apps that have symbols not common to the English language in their name.

Using the `ord()` function, we can identify the unicode of any character. Common English characters are encoded using the American Standard Code for Information Interchange (ASCII) standard. The common
ASCII characters have a corresponding code from 0 to 127. So, to remove the non-English apps, we'll want to remove apps containing characters with codes greater than 127.

In [12]:
## Check if string is English ##
def is_english(string):    
    for character in string:
        if ord(character) > 127:
                return False
            
    return True

print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
False


But what about app names with other special characters not included in the standard ASCII set, like ™ or emoji? For example:

In [13]:
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

print(ord('™'))
print(ord('😜'))

False
False
8482
128540


To account for this, instead of removing apps with _any_ character codes greater than 127, we can allow up to 3 special characters per app name. This isn't a perfect solution, but should work for our purposes. Using this method, we should be able to identify and remove the majority of non-English apps, while keeping most of the English apps with special characters.

In [14]:
## Check if string is English, allowing 3 special chars ##
def is_english(string):
    non_ascii = 0
    
    for character in string:
        if ord(character) > 127:
            non_ascii += 1
            
            if non_ascii > 3:
                return False
            
    return True

Now that we've written a function to identify the English apps, let's use it to clean our datasets and store the results in new `_english` lists.

In [15]:
## Identify and Store English Apps ##
android_english = []
ios_english = []

for app in android_clean:
    name = app[0]
    if is_english(name):
        android_english.append(app)
        
for app in ios:
    name = app[1]
    if is_english(name):
        ios_english.append(app)

print('Sample of Android Dataset: \n---------------------------')
explore_data(android_english,0,3,True)
print('\n')
print('Sample of iOS Dataset: \n---------------------------')
explore_data(ios_english,0,3,True)

Sample of Android Dataset: 
---------------------------
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


Number of rows: 9614
Number of columns: 13


Sample of iOS Dataset: 
---------------------------
['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']




Exploring the data, we can see we now have 9614 Android apps and 6183 iOS apps remaining.

### 4) Removing Paid Apps <a id="paid"></a>

Finally, since our company builds only free apps, we will remove any paid apps from the datasets.

For the `ios` dataset, the app price is located at index `4`. In the `android` dataset, it looks like apps are categorized already as free or paid at index `6`. Let's take a closer look at what unique values exist in these columns.

In [16]:
## Freq Table for iOS Prices ##
ios_prices = {}
for app in ios_english:
    price = app[4]
    if price in ios_prices:
        ios_prices[price] += 1
    else:
        ios_prices[price] = 1

## Freq Table for Android Prices ##        
android_prices = {}
for app in android_english:
    price = app[6]
    if price in android_prices:
        android_prices[price] += 1
    else:
        android_prices[price] = 1

print('iOS prices:')
print(ios_prices)
print('\n')
print('Android prices:')
print(android_prices)

iOS prices:
{'0.0': 3222, '1.99': 610, '0.99': 641, '6.99': 165, '2.99': 669, '7.99': 30, '4.99': 375, '9.99': 76, '3.99': 266, '8.99': 8, '5.99': 43, '14.99': 15, '13.99': 6, '19.99': 13, '17.99': 3, '15.99': 4, '24.99': 8, '20.99': 1, '29.99': 6, '12.99': 1, '39.99': 2, '74.99': 1, '16.99': 2, '249.99': 1, '11.99': 3, '27.99': 1, '49.99': 2, '59.99': 3, '22.99': 1, '18.99': 1, '99.99': 1, '21.99': 1, '34.99': 1, '299.99': 1}


Android prices:
{'Free': 8863, 'Paid': 750, 'NaN': 1}


There are 3222 free iOS apps and 8863 free Android apps. Let's isolate the free apps and store them in a new list for each platform. These will be our final clean datasets that we will use for analysis.

In [17]:
ios_final = []
android_final = []

## Isolating Free iOS Apps ##
for app in ios_english:
    price = app[4]
    if price == '0.0':
        ios_final.append(app)
        
## Isolating Free Android Apps ##        
android_prices = {}
for app in android_english:
    price = app[6]
    if price == 'Free':
        android_final.append(app)

print('Number Android Apps Remaining', ':', len(android_final))
print('Number iOS Apps Remaining', ':', len(ios_final))

Number Android Apps Remaining : 8863
Number iOS Apps Remaining : 3222


At last, we're ready to start analysis!

## Analyzing The Data <a id="analyzing"></a>

Now that we have a clean set of data for each app store, let's revisit our intent in analyzing the data and what questions we hope to answer.

Our company, Evergreen Apps LLC, builds free iOS and Android apps, where the main source of revenue is in-app ads. To minimize risks and overhead when building products, the company validates app ideas by:

1. Building a minimal Android version and adding it to Google Play.
2. If the app has a good response from users, then develop it further.
3. If the app is profitable after six months, build an iOS version and add it to the iOS App Store.

Since our strategy involves building apps that will succeed in both markets, we'll aim to identify what types of apps are viable in both iOS and Android markets.

### Most Common Genres <a id="common"></a>

First, let's identify what genres are most common. In the iOS dataset, we can use the `prime_genre` column. In the Android dataset, we can try both the `Category` and `Genres` columns.

#### Part 1: Building frequency tables
We will need to use frequency tables to analyze the data from each column. Let's write a function that will create frequency tables for us.

In [18]:
def freq_table(dataset, index):
    table = {}
    total = 0
    
    # Store frequency of each value in column
    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
    
    # Calculate percentage of total apps for each value
    table_percentages = {}
    for key in table:
        percentage = round((table[key] / total) * 100,2)
        table_percentages[key] = percentage
        
    return table_percentages

In [19]:
freq_table(ios_final, 11)

{'Social Networking': 3.29,
 'Photo & Video': 4.97,
 'Games': 58.16,
 'Music': 2.05,
 'Reference': 0.56,
 'Health & Fitness': 2.02,
 'Weather': 0.87,
 'Utilities': 2.51,
 'Travel': 1.24,
 'Shopping': 2.61,
 'News': 1.33,
 'Navigation': 0.19,
 'Lifestyle': 1.58,
 'Entertainment': 7.88,
 'Food & Drink': 0.81,
 'Sports': 2.14,
 'Book': 0.43,
 'Finance': 1.12,
 'Education': 3.66,
 'Productivity': 1.74,
 'Business': 0.53,
 'Catalogs': 0.12,
 'Medical': 0.19}

#### Part 2: Ordering frequency tables

Above we can see what percentage of all apps are in each genre, but the results are difficult to digest in the current order. Let's build a second function that will display frequency tables in order from most to least.

In [20]:
def display_table(dataset,index):
    table = freq_table(dataset,index)
    
    # Store key-value pairs as tuples
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
        
    # Sort and print the tuples
    table_sorted = sorted(table_display, reverse = True)
    
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

#### Part 3: Examining frequency tables

Using our new function, let's first examine the `prime_genres` column from the iOS dataset.

In [21]:
display_table(ios_final, 11)

Games : 58.16
Entertainment : 7.88
Photo & Video : 4.97
Education : 3.66
Social Networking : 3.29
Shopping : 2.61
Utilities : 2.51
Sports : 2.14
Music : 2.05
Health & Fitness : 2.02
Productivity : 1.74
Lifestyle : 1.58
News : 1.33
Travel : 1.24
Finance : 1.12
Weather : 0.87
Food & Drink : 0.81
Reference : 0.56
Business : 0.53
Book : 0.43
Navigation : 0.19
Medical : 0.19
Catalogs : 0.12


We can see that **Games** are by far the most common app genre, comprising over half (58%!) of all the free English apps on the iOS App Store. The second most common genre is **Entertainment**, though this makes up a much smaller percentage at less than 8%. Third most common are **Photo & Video** apps, closely following at a little less than 5%. 

Beyond that is Education (3.66%), Social Networking (3.29%), Shopping (2.61%), Utilities (2.51%), and Sports (2.14%). These eight categories combined make up 85% of the total free English apps on the iOS App Store. 

Our general impression is that apps designed for fun and entertainment dominate the iOS App Store -- while apps built for a more practical purpose such as reference, business, or news, are far less commonly built.

Next let's examine the most common `Genres` in the Android dataset.

In [22]:
display_table(android_final, 9) # Genres Column

Tools : 8.45
Entertainment : 6.07
Education : 5.35
Business : 4.59
Productivity : 3.89
Lifestyle : 3.89
Finance : 3.7
Medical : 3.53
Sports : 3.46
Personalization : 3.32
Communication : 3.24
Action : 3.1
Health & Fitness : 3.08
Photography : 2.94
News & Magazines : 2.8
Social : 2.66
Travel & Local : 2.32
Shopping : 2.25
Books & Reference : 2.14
Simulation : 2.04
Dating : 1.86
Arcade : 1.85
Video Players & Editors : 1.77
Casual : 1.76
Maps & Navigation : 1.4
Food & Drink : 1.24
Puzzle : 1.13
Racing : 0.99
Role Playing : 0.94
Libraries & Demo : 0.94
Auto & Vehicles : 0.93
Strategy : 0.9
House & Home : 0.82
Weather : 0.8
Events : 0.71
Adventure : 0.68
Comics : 0.61
Beauty : 0.6
Art & Design : 0.6
Parenting : 0.5
Card : 0.45
Casino : 0.43
Trivia : 0.42
Educational;Education : 0.39
Board : 0.38
Educational : 0.37
Education;Education : 0.34
Word : 0.26
Casual;Pretend Play : 0.24
Music : 0.2
Racing;Action & Adventure : 0.17
Puzzle;Brain Games : 0.17
Entertainment;Music & Video : 0.17
Casual;B


The `Genres` column in the Android dataset seems to provide a very granular and overwhelming number of different genres. Let's try taking a look at the `Category` column instead to see if it provides a more general categorization scheme.

In [23]:
display_table(android_final, 1) # Category Column

FAMILY : 18.9
GAME : 9.73
TOOLS : 8.46
BUSINESS : 4.59
LIFESTYLE : 3.9
PRODUCTIVITY : 3.89
FINANCE : 3.7
MEDICAL : 3.53
SPORTS : 3.4
PERSONALIZATION : 3.32
COMMUNICATION : 3.24
HEALTH_AND_FITNESS : 3.08
PHOTOGRAPHY : 2.94
NEWS_AND_MAGAZINES : 2.8
SOCIAL : 2.66
TRAVEL_AND_LOCAL : 2.34
SHOPPING : 2.25
BOOKS_AND_REFERENCE : 2.14
DATING : 1.86
VIDEO_PLAYERS : 1.79
MAPS_AND_NAVIGATION : 1.4
FOOD_AND_DRINK : 1.24
EDUCATION : 1.16
ENTERTAINMENT : 0.96
LIBRARIES_AND_DEMO : 0.94
AUTO_AND_VEHICLES : 0.93
HOUSE_AND_HOME : 0.82
WEATHER : 0.8
EVENTS : 0.71
PARENTING : 0.65
ART_AND_DESIGN : 0.64
COMICS : 0.62
BEAUTY : 0.6


Much better!

**Family** apps seem to be the most common on the Android App store. However, when we take a look at the family category on the actual Google Play store, it seems that most of the family apps are **Games** for family or kids. Together then, this would bring Family and Games to 28.63% of all free English apps on the Android app store.

Although Family and Games comprise the top categories on the Android App Store, that's only half the amount of games on the iOS App Store. The Android App Store seems to contain a much more balanced app landscape, with the next most common categories being more practical apps such as **Tools** (8.46%), **Business** (4.59%), **Lifestyle** (3.9%), **Productivity** (3.89%), and **Finance** (3.7%).

Okay, so far we have an idea of the most commonly *built* free English apps in each market, but we don't know much about the *demand* for those apps. For example, there are tons of games apps available on both platforms, but does the demand from actual users warrant that many, or do they go largely unplayed?

### Most Popular Genres <a id="popular"></a>

One way to better understand demand for different app genres is to assess which genres attract the most users. 

#### Part 1: Most Popular Genres on iOS

The Android dataset contains data on number of `Installs` per app, however the iOS dataset doesn't have such a direct data point to go from. So for iOS, let's instead try analyzing genre popularity by the **total number of ratings** (`rating_count_tot`).

In [24]:
ios_genres = freq_table(ios_final,11) # Get list of iOS app genres
ios_avg_sorted = []

for genre in ios_genres:
    total = 0
    len_genre = 0
    
    # Store total ratings per genre
    for app in ios_final:
        genre_app = app[11]
        if genre_app == genre:
            total += int(app[5])
            len_genre += 1 # Count total apps per genre
    
    # Find average ratings for each genre
    ios_avg_sorted.append((round(total / len_genre),genre))

# Sort averages descending
ios_avg_sorted = sorted(ios_avg_sorted, reverse = True)

for value in ios_avg_sorted:
    print(value[1],':',value[0])

Navigation : 86090
Reference : 74942
Social Networking : 71548
Music : 57327
Weather : 52280
Book : 39758
Food & Drink : 33334
Finance : 31468
Photo & Video : 28442
Travel : 28244
Shopping : 26920
Health & Fitness : 23298
Sports : 23009
Games : 22789
News : 21248
Productivity : 21028
Utilities : 18684
Lifestyle : 16486
Entertainment : 14030
Business : 7491
Education : 7004
Catalogs : 4004
Medical : 612


Above we have a list of average number of ratings per app in each genre. Next, let's write a function to dig into the data for each genre:

In [25]:
def print_ios_genre(genre,results=12): # Displays top 12 rows by default
    apps = 0
    
    for app in ios_final:
        if app[11] == genre and apps < results:
            print(app[1], ':', app[5])
            apps += 1
            
print_ios_genre('Navigation')

Waze - GPS Navigation, Maps & Real-time Traffic : 345046
Google Maps - Navigation & Transit : 154911
Geocaching® : 12811
CoPilot GPS – Car Navigation & Offline Maps : 3582
ImmobilienScout24: Real Estate Search in Germany : 187
Railway Route Search : 5


Intersting results! 

Among free English apps on iOS, **Navigation** apps appear to have the most ratings per app. However, on closer inspection we can see the results are skewed significantly. There are only 6 apps in this genre, and the vast majority of ratings are for two extremely popular navigation apps: Waze and Google Maps. These would be extremely difficult to compete with. 

Let's investigate other genres.

In [26]:
print_ios_genre('Social Networking',18)

Facebook : 2974676
Pinterest : 1061624
Skype for iPhone : 373519
Messenger : 351466
Tumblr : 334293
WhatsApp Messenger : 287589
Kik : 260965
ooVoo – Free Video Call, Text and Voice : 177501
TextNow - Unlimited Text + Calls : 164963
Viber Messenger – Text & Call : 164249
Followers - Social Analytics For Instagram : 112778
MeetMe - Chat and Meet New People : 97072
We Heart It - Fashion, wallpapers, quotes, tattoos : 90414
InsTrack for Instagram - Analytics Plus More : 85535
Tango - Free Video Call, Voice and Chat : 75412
LinkedIn : 71856
Match™ - #1 Dating App. : 60659
Skype for iPad : 60163


In [27]:
print_ios_genre('Music',18)

Pandora - Music & Radio : 1126879
Spotify Music : 878563
Shazam - Discover music, artists, videos & lyrics : 402925
iHeartRadio – Free Music & Radio Stations : 293228
SoundCloud - Music & Audio : 135744
Magic Piano by Smule : 131695
Smule Sing! : 119316
TuneIn Radio - MLB NBA Audiobooks Podcasts Music : 110420
Amazon Music : 106235
SoundHound Song Search & Music Player : 82602
Sonos Controller : 48905
Bandsintown Concerts : 30845
Karaoke - Sing Karaoke, Unlimited Songs! : 28606
My Mixtapez Music : 26286
Sing Karaoke Songs Unlimited with StarMaker : 26227
Ringtones for iPhone & Ringtone Maker : 25403
Musi - Unlimited Music For YouTube : 25193
AutoRap by Smule : 18202


The same applies to **Social Networking** and **Music**. 

Total ratings are skewed significantly by big players in these spaces, such as Facebook and Pinterest, or Pandora and Spotify. These would be tough spaces to compete in, where the big players appear to get the vast majority of users, and the other apps attract a *much* smaller number of ratings overall. 

In [28]:
print_ios_genre('Reference')

Bible : 985920
Dictionary.com Dictionary & Thesaurus : 200047
Dictionary.com Dictionary & Thesaurus for iPad : 54175
Google Translate : 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588
Merriam-Webster Dictionary : 16849
Night Sky : 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497
Guides for Pokémon GO - Pokemon GO News and Cheats : 826


The **Reference** genre could be a more viable option. Most ratings are for Bible and Dictionary.com. However, there are very few apps in this genre to compete with overall, and we can imagine use cases for other reference books that wouldn't compete with the ones listed here.

Let's keep looking for other genres that may have a more even playing field of competing companies or interesting market opportunities:

- **Weather** would be tough. The available apps serve similar use cases and are highly specialized.
- **Book** doesn't seem to have many ratings overall, with the majority going to 4 popular apps. Though, similar to Reference apps, there could be market opportunity around specific categories of books, for users seeking to consume a particular type of material.
- **Food & Drink** seems largely centered around delivery and chain restaurants. Not likely viable for us.
- **Finance** seems dominated by large banks and a couple other big players in finance tech.
- **Photo & Video** could be a possibility. The big players Instagram, Snapchat, and Youtube are at the top, but many other apps for a variety of purposes receive plenty of ratings as well. That said, we might need specialized knowledge to build photo and video apps.

*(Data for these genres can be found in the [appendix](#appendix).)*

#### Part 2: Most Popular Genres on Android

To analyze the most popular genres on Android, we can use data on the total number of installs. We hope this might offer a more relevant picture of popularity than total number of ratings, which we used for iOS.

In [29]:
display_table(android_final, 5) # Installs column

1,000,000+ : 15.73
100,000+ : 11.55
10,000,000+ : 10.55
10,000+ : 10.2
1,000+ : 8.39
100+ : 6.92
5,000,000+ : 6.83
500,000+ : 5.56
50,000+ : 4.77
5,000+ : 4.51
10+ : 3.54
500+ : 3.25
50,000,000+ : 2.3
100,000,000+ : 2.13
50+ : 1.92
5+ : 0.79
1+ : 0.51
500,000,000+ : 0.27
1,000,000,000+ : 0.23
0+ : 0.05


One challenge with this data is that the total number of installs is imprecise. Instead, each app is categorized within a general range. For our purposes though, we don't necessarily need a precise number to understand general popularity, so this shouldn't present an issue. We will just use each number as listed to gain a **general understanding of average installs per genre**.

Let's build a function to list average number of installs per genre in descending order.

In [30]:
android_genres = freq_table(android_final,1) # Get a list of android app genres
android_avg_sorted = []

for genre in android_genres:
    total = 0
    len_genre = 0
    
    # Store total installs per genre
    for app in android_final:
        genre_app = app[1]
        if genre_app == genre:
            n_installs = app[5]

            # Convert number installs to integer
            n_installs = n_installs.replace('+','')
            n_installs = n_installs.replace(',','')
            total += int(n_installs)
            
            len_genre += 1  # Count total apps per genre
    
    # Find average ratings for each genre
    android_avg_sorted.append((round(total / len_genre),genre))

# Sort and pring averages
android_avg_sorted = sorted(android_avg_sorted, reverse = True)

for value in android_avg_sorted:
    print(value[1],':',value[0])

COMMUNICATION : 38456119
VIDEO_PLAYERS : 24727872
SOCIAL : 23253652
PHOTOGRAPHY : 17840110
PRODUCTIVITY : 16787331
GAME : 15588016
TRAVEL_AND_LOCAL : 13984078
ENTERTAINMENT : 11640706
TOOLS : 10801391
NEWS_AND_MAGAZINES : 9549178
BOOKS_AND_REFERENCE : 8767812
SHOPPING : 7036877
PERSONALIZATION : 5201483
WEATHER : 5074486
HEALTH_AND_FITNESS : 4188822
MAPS_AND_NAVIGATION : 4056942
FAMILY : 3697848
SPORTS : 3638640
ART_AND_DESIGN : 1986335
FOOD_AND_DRINK : 1924898
EDUCATION : 1833495
BUSINESS : 1712290
LIFESTYLE : 1437816
FINANCE : 1387692
HOUSE_AND_HOME : 1331541
DATING : 854029
COMICS : 817657
AUTO_AND_VEHICLES : 647318
LIBRARIES_AND_DEMO : 638504
PARENTING : 542604
BEAUTY : 513152
EVENTS : 253542
MEDICAL : 120551


This list seems to represent a much broader landscape of viable categories than our iOS list. Let's take a look at the top genres.

In [31]:
## Print top most popular apps in a genre ##
def print_android_genre(genre,results=12):
    genre_list = []
    
    # Create list of each app with num installs, by genre
    for app in android_final:
        if app[1] == genre:
            n_installs = app[5]
            n_installs = n_installs.replace('+','')
            n_installs = n_installs.replace(',','')
            genre_list.append((int(n_installs),app[0]))
    
    # Sort list by num installs
    genre_list = sorted(genre_list, reverse=True)    
    
    for value in genre_list[0:results]:
        print(value[1],':',(f'{value[0]:,}')) # format num installs with commas
            
print_android_genre('COMMUNICATION',25)

WhatsApp Messenger : 1,000,000,000
Skype - free IM & video calls : 1,000,000,000
Messenger – Text and Video Chat for Free : 1,000,000,000
Hangouts : 1,000,000,000
Google Chrome: Fast & Secure : 1,000,000,000
Gmail : 1,000,000,000
imo free video calls and chat : 500,000,000
Viber Messenger : 500,000,000
UC Browser - Fast Download Private & Secure : 500,000,000
LINE: Free Calls & Messages : 500,000,000
Google Duo - High Quality Video Calls : 500,000,000
imo beta free calls and text : 100,000,000
Yahoo Mail – Stay Organized : 100,000,000
Who : 100,000,000
WeChat : 100,000,000
UC Browser Mini -Tiny Fast Private & Secure : 100,000,000
Truecaller: Caller ID, SMS spam blocking & Dialer : 100,000,000
Telegram : 100,000,000
Opera Mini - fast web browser : 100,000,000
Opera Browser: Fast and Secure : 100,000,000
Messenger Lite: Free Calls & Messages : 100,000,000
Kik : 100,000,000
KakaoTalk: Free Calls & Text : 100,000,000
GO SMS Pro - Messenger, Free Themes, Emoji : 100,000,000
Firefox Browser 

In [32]:
print_android_genre('VIDEO_PLAYERS')

YouTube : 1,000,000,000
Google Play Movies & TV : 1,000,000,000
MX Player : 500,000,000
VivaVideo - Video Editor & Photo Movie : 100,000,000
VideoShow-Video Editor, Video Maker, Beauty Camera : 100,000,000
VLC for Android : 100,000,000
Motorola Gallery : 100,000,000
Motorola FM Radio : 100,000,000
Dubsmash : 100,000,000
Vote for : 50,000,000
Vigo Video : 50,000,000
VMate : 50,000,000


In [33]:
print_android_genre('SOCIAL', 15)

Instagram : 1,000,000,000
Google+ : 1,000,000,000
Facebook : 1,000,000,000
Snapchat : 500,000,000
Facebook Lite : 500,000,000
VK : 100,000,000
Tumblr : 100,000,000
Tik Tok - including musical.ly : 100,000,000
Tango - Live Video Broadcast : 100,000,000
Pinterest : 100,000,000
LinkedIn : 100,000,000
Badoo - Free Chat & Dating App : 100,000,000
BIGO LIVE - Live Stream : 100,000,000
ooVoo Video Calls, Messaging & Stories : 50,000,000
Zello PTT Walkie Talkie : 50,000,000


The top 3 categories, **Communications, Video Players,** and **Social**, seem dominated by some huge players like WhatsApp, Google, and Facebook, skewing the data. These would all be tough spaces to compete in.

Let's keep looking through other top genres for viable opportunities:
- **Photography** has some big players on top, but installs are generally distributed across a variety of apps. The genre is saturated. There could be space for more photography utility apps, but this could be tough without specialized knowledge.
- **Productivity** is largely dominated by Google, Microsoft, Evernote, and a handful of other popular apps. Likely not an easy space to enter.
- **Games** is highly saturated, though this could be a viable genre as there are many apps with plenty of installs, and people are always looking for new games to play. Likewise for the **Family** genre.
- **Travel and Local** doesn't appear to have space or interest for apps outside of the big maps or search apps like Google, TripAdvisor, or Yelp.
- **Tools** seems to have a wide variety of installs for utility apps that provide a specific function. There could be space to invent more tools here. 


*(Data for these genres can be found in the [appendix](#appendix).)*

# Conclusion <a class="" id="conclusion" href=""></a>

Let's revisit our goals from the introduction one last time: 

> In this sample project, we are acting as Data Analysts for mock company Evergreen Apps, LLC. Evergreen Apps builds free Android and iOS apps. Their primary source of revenue is via in-app ads. Therefore, we want to build free apps which will attract the most users with frequent views to produce ad revenue.

### Key Questions & Criteria

During our analysis, a few key questions emerged:
- Do big players cover most opportunities for the genre?
- Are most opportunities for the genre covered by other apps?
- Does the genre get a lot of interest from users (i.e. many ratings or installs)?
- Does the genre require specialized knowledge that's difficult to acquire?

To maximize market opportunities for free English apps funded by in-app ads, we'll want to consider the following criteria:
- We don't want to compete with big players who dominate the market.
- We want to avoid genres where opportunities are already covered by other apps, or where really only 1 app of its kind is needed by a user.
- We want to find genres with a lot of broad interest from users.
- We want to avoid apps that would require specialized knowledge too expensive or time-consuming to reasonably come up to speed on.

### Viable Genres with Market Opportunities

We have identified 3 general categories that meet the  needs of Evergreen Apps, LLC:
- Games and family entertainment
- Utilities and tools
- Books, reference, and education

These categories may have some big players in them that cover general use cases, but they seem to have plenty of market opportunity for a variety of apps. Users are likely to use many different apps in each of these genres to meet every day needs. While some apps in these genres may have required specialized knowledge to build, that's generally a choice made by the specialists creating the apps, rather than a requirement of the genres themselves.

#### Next Steps

Next steps would be to explore each of these categories in further detail. There may be benefit in selecting one category to work within for the long run, where each new prototype built may benefit from work completed on previously built prototypes. When exploring each category, we should consider Evergreen Apps' current core competencies, as well as user needs and business goals, to select a category that meets the company's current knowledge, interests, and financial needs.

---
# Appendix <a id="appendix"></a>

### iOS – Most Popular Genres (By Total Ratings)

In [34]:
print_ios_genre('Weather')

The Weather Channel: Forecast, Radar & Alerts : 495626
The Weather Channel App for iPad – best local forecast, radar map, and storm tracking : 208648
WeatherBug - Local Weather, Radar, Maps, Alerts : 188583
MyRadar NOAA Weather Radar Forecast : 150158
AccuWeather - Weather for Life : 144214
Yahoo Weather : 112603
Weather Underground: Custom Forecast & Local Radar : 49192
NOAA Weather Radar - Weather Forecast & HD Radar : 45696
Weather Live Free - Weather Forecast & Alerts : 35702
Storm Radar : 22792
QuakeFeed Earthquake Map, Alerts, and News : 6081
Moji Weather - Free Weather Forecast : 2333


In [35]:
print_ios_genre('Book')

Kindle – Read eBooks, Magazines & Textbooks : 252076
Audible – audio books, original series & podcasts : 105274
Color Therapy Adult Coloring Book for Adults : 84062
OverDrive – Library eBooks and Audiobooks : 65450
HOOKED - Chat Stories : 47829
BookShout: Read eBooks & Track Your Reading Goals : 879
Dr. Seuss Treasury — 50 best kids books : 451
Green Riding Hood : 392
Weirdwood Manor : 197
MangaZERO - comic reader : 9
ikouhoushi : 0
MangaTiara - love comic reader : 0


In [36]:
print_ios_genre('Food & Drink')

Starbucks : 303856
Domino's Pizza USA : 258624
OpenTable - Restaurant Reservations : 113936
Allrecipes Dinner Spinner : 109349
DoorDash - Food Delivery : 25947
UberEATS: Uber for Food Delivery : 17865
Postmates - Food Delivery, Faster : 9519
Dunkin' Donuts - Get Offers, Coupons & Rewards : 9068
Chick-fil-A : 5665
McDonald's : 4050
Deliveroo: Restaurant Delivery - Order Food Nearby : 1702
SONIC Drive-In : 1645


In [37]:
print_ios_genre('Finance',20)

Chase Mobile℠ : 233270
Mint: Personal Finance, Budget, Bills & Money : 232940
Bank of America - Mobile Banking : 119773
PayPal - Send and request money safely : 119487
Credit Karma: Free Credit Scores, Reports & Alerts : 101679
Capital One Mobile : 56110
Citi Mobile® : 48822
Wells Fargo Mobile : 43064
Chase Mobile : 34322
Square Cash - Send Money for Free : 23775
Capital One for iPad : 21858
Venmo : 21090
USAA Mobile : 19946
TaxCaster – Free tax refund calculator : 17516
Amex Mobile : 11421
TurboTax Tax Return App - File 2016 income taxes : 9635
Bank of America - Mobile Banking for iPad : 7569
Wells Fargo for iPad : 2207
Stash Invest: Investing & Financial Education : 1655
Digit: Save Money Without Thinking About It : 1506


In [38]:
print_ios_genre('Photo & Video',30)

Instagram : 2161558
Snapchat : 323905
YouTube - Watch Videos, Music, and Live Streams : 278166
Pic Collage - Picture Editor & Photo Collage Maker : 123433
Funimate video editor: add cool effects to videos : 123268
musical.ly - your video social network : 105429
Photo Collage Maker & Photo Editor - Live Collage : 93781
Vine Camera : 90355
Google Photos - unlimited photo and video storage : 88742
Flipagram : 79905
Mixgram - Picture Collage Maker - Pic Photo Editor : 54282
Shutterfly: Prints, Photo Books, Cards Made Easy : 51427
Pic Jointer – Photo Collage, Camera Effects Editor : 51330
Color Pop Effects - Photo Editor & Picture Editing : 45320
Photo Grid - photo collage maker & photo editor : 40531
iSwap Faces LITE : 39722
MOLDIV - Photo Editor, Collage & Beauty Camera : 39501
Photo Editor by Aviary : 39501
Photo Lab: Picture Editor, effects & fun face app : 34585
Rookie Cam - Photo Editor & Filter Camera : 33921
FotoRus -Camera & Photo Editor & Pic Collage Maker : 32558
PicsArt Photo St

In [39]:
print_ios_genre('Travel')

Google Earth : 446185
Yelp - Nearby Restaurants, Shopping & Services : 223885
GasBuddy : 145549
TripAdvisor Hotels Flights Restaurants : 56194
Uber : 49466
Lyft : 46922
HotelTonight - Great Deals on Last Minute Hotels : 32341
Hotels & Vacation Rentals by Booking.com : 31261
Southwest Airlines : 30552
Airbnb : 22302
Expedia Hotels, Flights & Vacation Package Deals : 10278
Fly Delta : 8094


In [40]:
print_ios_genre('Shopping',30)

Groupon - Deals, Coupons & Discount Shopping App : 417779
eBay: Best App to Buy, Sell, Save! Online Shopping : 262241
Wish - Shopping Made Fun : 141960
shopkick - Shopping Rewards & Discounts : 130823
Amazon App: shop, scan, compare, and read reviews : 126312
Target : 108131
Zappos: shop shoes & clothes, fast free shipping : 103655
Walgreens – Pharmacy, Photo, Coupons and Shopping : 88885
Best Buy : 80424
Walmart: Free 2-Day Shipping,* Easy Store Shopping : 70286
OfferUp - Buy. Sell. Simple. : 57348
Apple Store : 55171
Shop Savvy Barcode Scanner - Price Compare & Deals : 54630
Ibotta: Cash Back App, Grocery Coupons & Shopping : 44313
letgo: Buy & Sell Second Hand Stuff : 38424
CVS Pharmacy : 35981
Victoria’s Secret – The Sexiest Bras & Lingerie : 34507
Etsy: Shop Handmade, Vintage & Creative Goods : 30434
Gilt : 26353
Mercari: Shopping Marketplace to Buy & Sell Stuff : 24244
Shopular Coupons, Weekly Deals for Target, Walmart : 22729
RetailMeNot Shopping Deals, Coupons, Savings : 18544


In [41]:
print_ios_genre('Health & Fitness',20)

Calorie Counter & Diet Tracker by MyFitnessPal : 507706
Lose It! – Weight Loss Program and Calorie Counter : 373835
Weight Watchers : 136833
Sleep Cycle alarm clock : 104539
Fitbit : 90496
Period Tracker Lite : 53620
Nike+ Training Club - Workouts & Fitness Plans : 33969
Plant Nanny - Water Reminder with Cute Plants : 27421
Sworkit - Custom Workouts for Exercise & Fitness : 16819
Clue Period Tracker: Period & Ovulation Tracker : 13436
Headspace : 12819
Fooducate - Lose Weight, Eat Healthy,Get Motivated : 11875
Runtastic Running, Jogging and Walking Tracker : 10298
WebMD for iPad : 9142
8fit - Workouts, meal plans and personal trainer : 8730
Garmin Connect™ Mobile : 8341
Record by Under Armour, connects with UA HealthBox : 7754
Fitstar Personal Trainer : 7496
My Cycles Period and Ovulation Tracker : 7469
Seven - 7 Minute Workout Training Challenge : 6808


In [42]:
print_ios_genre('Sports',30)

ESPN: Get scores, news, alerts & watch live sports : 290996
Yahoo Fantasy Sports : 190670
WatchESPN : 159735
The Masters Tournament : 148160
Yahoo Sports - Teams, Scores, News & Highlights : 137951
ESPN Fantasy Football Baseball Basketball Hockey : 64925
CBS Sports App - Sports Scores, News, Stats, Watch : 59639
FOX Sports Mobile : 57500
2016 U.S. Open Golf Championship : 54192
NBC Sports : 47172
NBA : 43682
ESPN Tournament Challenge : 39642
2016 US Open Tennis Championships : 37522
NFL : 27317
MLB.com At Bat : 21830
The Championships, Wimbledon 2016 - Tennis Grand Slam : 20953
DraftKings - Daily Fantasy Golf, Baseball, & More : 20251
Bleacher Report: Sports news, scores, & highlights : 16979
Univision Deportes: Liga MX, MLS, Fútbol En Vivo : 16683
NASCAR MOBILE : 16385
NHL : 15554
FOX Sports GO : 13898
NCAA March Madness Live - Men's College Basketball : 13572
PGA Championship 2016 – Baltusrol Golf Club : 10472
365Scores : 9879
NFL Fantasy Football - Official NFL Fantasy App : 8234
Fa

In [43]:
print_ios_genre('Games',100)

Clash of Clans : 2130805
Temple Run : 1724546
Candy Crush Saga : 961794
Angry Birds : 824451
Subway Surfers : 706110
Solitaire : 679055
CSR Racing : 677247
Crossy Road - Endless Arcade Hopper : 669079
Injustice: Gods Among Us : 612532
Hay Day : 567344
PAC-MAN : 508808
DragonVale : 503230
Head Soccer : 481564
Despicable Me: Minion Rush : 464312
The Sims™ FreePlay : 446880
Sonic Dash : 418033
8 Ball Pool™ : 416736
Tiny Tower - Free City Building : 414803
Jetpack Joyride : 405647
Bike Race - Top Motorcycle Racing Games : 405007
Kim Kardashian: Hollywood : 397730
Trivia Crack : 393469
WordBrain : 391401
Sniper 3D Assassin: Shoot to Kill Gun Game : 386521
Flow Free : 373857
Geometry Dash Lite : 370370
▻Sudoku : 359832
Fruit Ninja® : 327025
Pixel Gun 3D : 301182
Temple Run 2 : 295211
My Horse : 293857
Word Cookies! : 287095
Dragon City Mobile : 277268
The Simpsons™: Tapped Out : 274501
Plants vs. Zombies™ 2 : 267394
Clash Royale : 266921
Pokémon GO : 257627
CSR Racing 2 : 257100
Star Wars™: 

In [44]:
print_ios_genre('News')

Twitter : 354058
Fox News : 132703
CNN: Breaking US & World News, Live Video : 112886
Reddit Official App: All That's Trending and Viral : 67560
USA TODAY : 61724
ABC News - US & World News + Live Video : 48407
NBC News : 32881
HuffPost - News, Politics & Entertainment : 29107
The Washington Post Classic : 18572
WIRED Magazine : 12074
CBS News - Watch Free Live Breaking News : 11691
The Guardian : 8176


In [45]:
print_ios_genre('Productivity',30)

Evernote - stay organized : 161065
Gmail - email by Google: secure, fast & organized : 135962
iTranslate - Language Translator & Dictionary : 123215
Yahoo Mail - Keeps You Organized! : 113709
Google Docs : 64259
Google Drive - free online storage : 59255
Dropbox : 49578
Microsoft Word : 47999
Microsoft OneNote : 39638
Microsoft Outlook - email and calendar : 32807
Hotspot Shield Free VPN Proxy & Wi-Fi Privacy : 32499
Documents 6 - File manager, PDF reader and browser : 29110
Google Sheets : 24602
Microsoft Excel : 24430
Inbox by Gmail : 21561
T-Mobile : 19977
Paper by FiftyThree - Sketch, Diagram, Take Notes : 18219
MyScript Calculator - Handwriting calculator : 16555
VPN Proxy Master - Unlimited WiFi security VPN : 13674
Microsoft OneDrive – File & photo cloud storage : 12797
Ever - Capture Your Memories : 12755
Speak & Translate － Voice and Text Translator : 12062
Tayasui Sketches : 11505
Drawing Desk - Draw, Paint, Doodle & Sketch board : 11040
Microsoft PowerPoint : 10939
Email - F

In [46]:
print_ios_genre('Utilities',20)

Google – Search made just for mobile : 479440
Flashlight Ⓞ : 130450
My Verizon : 126948
myAT&T : 108507
The Calculator - Free and Easy Calculating! : 99244
Speedtest by Ookla : 65016
Google Chrome – The Fast and Secure Web Browser : 55750
Alarm Clock HD Free - Digital Alarm Clock Display : 54496
Calculator Pro for iPad Free - Smart Calculator : 47803
Bitmoji - Your Personal Emoji : 37972
Calculator‰ : 36879
Flashlight ！ : 35769
Truecaller - Spam Identification & Block : 27791
Browser and File Manager for Documents : 27750
Flashlight for iPhone , iPod and iPad : 26697
RainbowKey - Color keyboard themes, fonts & GIF : 23063
Calculator HD Free : 14986
QR Reader for iPhone : 12683
Puffin Web Browser : 11677
Evolution Calculator - CP & XP - for Pokemon GO! : 9269


In [47]:
print_ios_genre('Lifestyle')

Zillow Real Estate - Homes for Sale & for Rent : 342969
Tinder : 143040
Text Free: Free Texting + Calling + MMS : 100477
Countdown‼ (Event Reminders and Timer) : 60490
PINK Nation : 49816
happn — Dating app — Find and meet your crush : 20546
Perfect365 - Custom makeup designs and beauty tips : 19540
ipsy - Makeup, subscription and beauty tips : 17489
cute icon&wallpaper dressup - CocoPPa : 12508
Bumble – Find a Date, Meet Friends & Network : 10109
IKEA Catalog : 8939
Monogram - Wallpaper & Backgrounds Maker HD DIY with Glitter Themes : 7427


In [48]:
print_ios_genre('Entertainment',30)

Netflix : 308844
Fandango Movies - Times + Tickets : 291787
Colorfy: Coloring Book for Adults : 247809
IMDb Movies & TV - Trailers and Showtimes : 183425
TRUTH or DARE!!! - FREE : 171055
Mad Libs : 117889
Twitch : 109549
Action Movie FX : 101222
Voice Changer Plus : 98777
iFunny :) : 98344
The CW : 97368
The Moron Test : 88613
DIRECTV : 81006
ABC – Watch Live TV & Stream Full Episodes : 78890
Xbox : 72187
Redbox : 60236
Talking Tom Cat 2 for iPad : 56399
Hulu: Watch TV Shows & Stream the Latest Movies : 56170
NBC – Watch Now and Stream Full TV Episodes : 55950
Emoji> : 55338
DIRECTV App for iPad : 47506
Amazon Prime Video : 43667
CBS Full Episodes and Live TV : 39436
FOX NOW - Watch Full Episodes and Stream Live TV : 39391
Talking Angela for iPad : 32763
Recolor - Coloring Book : 31180
Talking Ben the Dog for iPad : 31116
Talking Tom Cat for iPad : 29492
YouTube Kids : 28560
Tom's Love Letters : 27711


In [49]:
print_ios_genre('Business')

Indeed Job Search : 38681
Flashlight ◎ : 24744
Adobe Acrobat Reader: View, Create, & Convert PDFs : 20069
Scanner App - PDF Document Scan : 11696
SayHi Translate : 8623
ADP Mobile Solutions : 8324
Sideline - 2nd Phone Number : 7907
Uber Driver : 3289
AirWatch Agent : 1150
VPN Go - Safe Fast & Stable VPN Proxy : 881
Cisco AnyConnect : 825
GreenVPN - Free & fast VPN with unlimited traffic : 464


In [50]:
print_ios_genre('Education',20)

Duolingo - Learn Spanish, French and more : 162701
Guess My Age  Math Magic : 123190
Lumosity - Brain Training : 96534
Elevate - Brain Training and Games : 58092
Fit Brains Trainer : 46363
ClassDojo : 35440
Memrise: learn languages : 20383
Peak - Brain Training : 20322
Canvas by Instructure : 19981
ABCmouse.com - Early Learning Academy : 18749
Quizlet: Study Flashcards, Languages & Vocabulary : 16683
Photomath - Camera Calculator : 16523
iTunes U : 15801
Blackboard Mobile Learn™ : 13567
Star Chart : 13482
Remind: Fast, Efficient School Messaging : 9796
PBS KIDS Video : 8651
Toca Kitchen Monsters : 8062
Toca Hair Salon - Christmas Gift : 8049
Edmodo : 7197


In [51]:
print_ios_genre('Catalogs')

CPlus for Craigslist app - mobile classifieds : 13345
DRAGONS MODS FREE for Minecraft PC Game Edition : 2027
Face Swap and Copy Free – Switch & Fusion Faces in a Photo : 431
Ringtone Remixes - Marimba Remix Ringtones : 213


In [52]:
print_ios_genre('Medical')

SnoreLab : Record Your Snoring : 1341
Blink Health : 1198
Complete Anatomy : 1104
MyDx One: Find a Strain that Works for You : 29
iSleeping by iSommeil SARL : 0
Malodo : Stop au mal de dos : 0


### Android – Most Popular Genres (By Total Installs)

In [53]:
print_android_genre('PHOTOGRAPHY',40)

Google Photos : 1,000,000,000
Z Camera - Photo Editor, Beauty Selfie, Collage : 100,000,000
YouCam Perfect - Selfie Photo Editor : 100,000,000
YouCam Makeup - Magic Selfie Makeovers : 100,000,000
Sweet Selfie - selfie camera, beauty cam, photo edit : 100,000,000
S Photo Editor - Collage Maker , Photo Collage : 100,000,000
Retrica : 100,000,000
PicsArt Photo Studio: Collage Maker & Pic Editor : 100,000,000
PhotoGrid: Video & Pic Collage Maker, Photo Editor : 100,000,000
Photo Editor Pro : 100,000,000
Photo Editor Collage Maker Pro : 100,000,000
Photo Collage Editor : 100,000,000
LINE Camera - Photo editor : 100,000,000
Cymera Camera- Photo Editor, Filter,Collage,Layout : 100,000,000
Candy Camera - selfie, beauty camera, photo editor : 100,000,000
Camera360: Selfie Photo Editor with Funny Sticker : 100,000,000
BeautyPlus - Easy Photo Editor & Selfie Camera : 100,000,000
B612 - Beauty & Filter Camera : 100,000,000
AR effect : 100,000,000
Video Editor Music,Cut,No Crop : 50,000,000
VSCO : 

In [54]:
print_android_genre('PRODUCTIVITY',40)

Google Drive : 1,000,000,000
Microsoft Word : 500,000,000
Google Calendar : 500,000,000
Dropbox : 500,000,000
Cloud Print : 500,000,000
WPS Office - Word, Docs, PDF, Note, Slide & Sheet : 100,000,000
SwiftKey Keyboard : 100,000,000
Samsung Notes : 100,000,000
Microsoft PowerPoint : 100,000,000
Microsoft Outlook : 100,000,000
Microsoft OneNote : 100,000,000
Microsoft OneDrive : 100,000,000
Microsoft Excel : 100,000,000
Google Slides : 100,000,000
Google Sheets : 100,000,000
Google Keep : 100,000,000
Google Docs : 100,000,000
Evernote – Organizer, Planner for Notes & Memos : 100,000,000
ES File Explorer File Manager : 100,000,000
ColorNote Notepad Notes : 100,000,000
CamScanner - Phone PDF Creator : 100,000,000
Adobe Acrobat Reader : 100,000,000
myAT&T : 50,000,000
Verizon Cloud : 50,000,000
QR Droid : 50,000,000
My Airtel-Online Recharge, Pay Bill, Wallet, UPI : 50,000,000
Mobizen Screen Recorder - Record, Capture, Edit : 50,000,000
MEGA : 50,000,000
File Browser by Astro (File Manager)

In [55]:
print_android_genre('GAME',100)

Subway Surfers : 1,000,000,000
Temple Run 2 : 500,000,000
Pou : 500,000,000
My Talking Tom : 500,000,000
Candy Crush Saga : 500,000,000
slither.io : 100,000,000
Zombie Tsunami : 100,000,000
Yes day : 100,000,000
Vector : 100,000,000
Trivia Crack : 100,000,000
Traffic Racer : 100,000,000
Temple Run : 100,000,000
Talking Tom Gold Run : 100,000,000
Super Mario Run : 100,000,000
Sonic Dash : 100,000,000
Sniper 3D Gun Shooter: Free Shooting Games - FPS : 100,000,000
Smash Hit : 100,000,000
Skater Boy : 100,000,000
Shadow Fight 2 : 100,000,000
Score! Hero : 100,000,000
Roll the Ball® - slide puzzle : 100,000,000
Pokémon GO : 100,000,000
Plants vs. Zombies FREE : 100,000,000
Piano Tiles 2™ : 100,000,000
PAC-MAN : 100,000,000
My Talking Angela : 100,000,000
Modern Combat 5: eSports FPS : 100,000,000
Mobile Legends: Bang Bang : 100,000,000
Lep's World 2 🍀🍀 : 100,000,000
Jetpack Joyride : 100,000,000
Hungry Shark Evolution : 100,000,000
Hill Climb Racing 2 : 100,000,000
Hill Climb Racing : 100,0

In [56]:
print_android_genre('TRAVEL_AND_LOCAL')

Maps - Navigate & Explore : 1,000,000,000
Google Street View : 1,000,000,000
TripAdvisor Hotels Flights Restaurants Attractions : 100,000,000
Google Earth : 100,000,000
Booking.com Travel Deals : 100,000,000
trivago: Hotels & Travel : 50,000,000
VZ Navigator : 50,000,000
MAPS.ME – Offline Map and Travel Navigation : 50,000,000
2GIS: directory & navigator : 50,000,000
easyJet: Travel App : 10,000,000
Yelp: Food, Shopping, Services Nearby : 10,000,000
Yatra - Flights, Hotels, Bus, Trains & Cabs : 10,000,000


In [57]:
print_android_genre('ENTERTAINMENT')

Talking Ben the Dog : 100,000,000
Talking Angela : 100,000,000
Netflix : 100,000,000
IMDb Movies & TV : 100,000,000
Hotstar : 100,000,000
Twitch: Livestream Multiplayer Games & Esports : 50,000,000
Talking Ginger 2 : 50,000,000
PlayStation App : 50,000,000
Amazon Prime Video : 50,000,000
ivi - movies and TV shows in HD : 10,000,000
WWE : 10,000,000
Vudu Movies & TV : 10,000,000


In [58]:
print_android_genre('TOOLS',50)

Google : 1,000,000,000
Security Master - Antivirus, VPN, AppLock, Booster : 500,000,000
SHAREit - Transfer & Share : 500,000,000
Google Translate : 500,000,000
Gboard - the Google Keyboard : 500,000,000
Clean Master- Space Cleaner & Antivirus : 500,000,000
Tiny Flashlight + LED : 100,000,000
Speedtest by Ookla : 100,000,000
Share Music & Transfer Files - Xender : 100,000,000
Samsung Smart Switch Mobile : 100,000,000
Samsung Calculator : 100,000,000
Lookout Security & Antivirus : 100,000,000
Google Now Launcher : 100,000,000
Google Korean Input : 100,000,000
GO Keyboard - Cute Emojis, Themes and GIFs : 100,000,000
Device Help : 100,000,000
DU Battery Saver - Battery Charger & Battery Life : 100,000,000
Calculator : 100,000,000
Cache Cleaner-DU Speed Booster (booster & cleaner) : 100,000,000
CM Locker - Security Lockscreen : 100,000,000
Battery Doctor-Battery Life Saver & Battery Cooler : 100,000,000
Avast Mobile Security 2018 - Antivirus & App Lock : 100,000,000
Automatic Call Recorder 

In [59]:
print_android_genre('NEWS_AND_MAGAZINES')

Google News : 1,000,000,000
Twitter : 500,000,000
Flipboard: News For Our Time : 500,000,000
Dailyhunt (Newshunt) - Latest News, Viral Videos : 50,000,000
detikcom - Latest & Most Complete News : 10,000,000
Updates for Samsung - Android Update Versions : 10,000,000
Topbuzz: Breaking News, Videos & Funny GIFs : 10,000,000
SmartNews: Breaking News Headlines : 10,000,000
Reddit: Social News, Trending Memes & Funny Videos : 10,000,000
Read- Latest News, Information, Gossip and Politics : 10,000,000
Pulse Nabd - World News, Urgent : 10,000,000
Pocket : 10,000,000


In [60]:
print_android_genre('BOOKS_AND_REFERENCE',20)

Google Play Books : 1,000,000,000
Wattpad 📖 Free Books : 100,000,000
Bible : 100,000,000
Audiobooks from Audible : 100,000,000
Amazon Kindle : 100,000,000
Wikipedia : 10,000,000
Spanish English Translator : 10,000,000
Quran for Android : 10,000,000
Oxford Dictionary of English : Free : 10,000,000
NOOK: Read eBooks & Magazines : 10,000,000
Moon+ Reader : 10,000,000
JW Library : 10,000,000
HTC Help : 10,000,000
FBReader: Favorite Book Reader : 10,000,000
English Hindi Dictionary : 10,000,000
English Dictionary - Offline : 10,000,000
Dictionary.com: Find Definitions for English Words : 10,000,000
Dictionary - Merriam-Webster : 10,000,000
Dictionary : 10,000,000
Cool Reader : 10,000,000


In [61]:
print_android_genre('PERSONALIZATION',15)

ZEDGE™ Ringtones & Wallpapers : 100,000,000
Parallel Space - Multiple accounts & Two face : 100,000,000
Hola Launcher- Theme,Wallpaper : 100,000,000
GO Launcher - 3D parallax Themes & HD Wallpapers : 100,000,000
GO Keyboard - Emoticon keyboard, Free Theme, GIF : 100,000,000
CM Launcher 3D - Theme, Wallpapers, Efficient : 100,000,000
Backgrounds HD (Wallpapers) : 100,000,000
APUS Launcher - Theme, Wallpaper, Hide Apps : 100,000,000
ZenUI Launcher : 50,000,000
Yandex Browser with Protect : 50,000,000
Ringtone Maker : 50,000,000
Nova Launcher : 50,000,000
Koi Free Live Wallpaper : 50,000,000
icon wallpaper dressup💞CocoPPa : 10,000,000
iKeyboard - emoji, emoticons : 10,000,000


In [62]:
print_android_genre('SHOPPING',15)

eBay: Buy & Sell this Summer - Discover Deals Now! : 100,000,000
Wish - Shopping Made Fun : 100,000,000
Flipkart Online Shopping App : 100,000,000
Amazon Shopping : 100,000,000
AliExpress - Smarter Shopping, Better Living : 100,000,000
letgo: Buy & Sell Used Stuff, Cars & Real Estate : 50,000,000
The birth : 50,000,000
OLX - Buy and Sell : 50,000,000
Myntra Online Shopping App : 50,000,000
Mercado Libre: Find your favorite brands : 50,000,000
Lazada - Online Shopping & Deals : 50,000,000
Groupon - Shop Deals, Discounts & Coupons : 50,000,000
eBay Kleinanzeigen for Germany : 10,000,000
cPro Marketplace: Buy. Sell. Rent. Date. Jobs. : 10,000,000
ZALORA Fashion Shopping : 10,000,000


In [63]:
print_android_genre('WEATHER')

Weather & Clock Widget for Android : 50,000,000
The Weather Channel: Rain Forecast & Storm Alerts : 50,000,000
GO Weather - Widget, Theme, Wallpaper, Efficient : 50,000,000
AccuWeather: Daily Forecast & Live Weather Reports : 50,000,000
wetter.com - Weather and Radar : 10,000,000
Yandex.Weather : 10,000,000
Yahoo Weather : 10,000,000
Weather –Simple weather forecast : 10,000,000
Weather by WeatherBug: Forecast, Radar & Alerts : 10,000,000
Weather 14 Days : 10,000,000
Weather : 10,000,000
MyRadar NOAA Weather Radar : 10,000,000


In [64]:
print_android_genre('HEALTH_AND_FITNESS')

Samsung Health : 500,000,000
Period Tracker - Period Calendar Ovulation Tracker : 100,000,000
Calorie Counter - MyFitnessPal : 50,000,000
Workout Trainer: fitness coach : 10,000,000
Water Drink Reminder : 10,000,000
Strava Training: Track Running, Cycling & Swimming : 10,000,000
Step Counter - Pedometer Free & Calorie Counter : 10,000,000
Six Pack in 30 Days - Abs Workout : 10,000,000
Runtastic Running App & Mile Tracker : 10,000,000
Runkeeper - GPS Track Run Walk : 10,000,000
Period Tracker Clue: Period and Ovulation Tracker : 10,000,000
Period Tracker : 10,000,000


In [65]:
print_android_genre('MAPS_AND_NAVIGATION')

Waze - GPS, Maps, Traffic Alerts & Live Navigation : 100,000,000
Uber : 100,000,000
GPS Navigation & Offline Maps Sygic : 50,000,000
Free GPS Navigation : 50,000,000
Yandex.Transport : 10,000,000
Yahoo! transit guide free timetable, operation information, transfer search : 10,000,000
TomTom GPS Navigation Traffic : 10,000,000
Subway Terminator: Smarter Subway : 10,000,000
Moovit: Bus Time & Train Time Live Info : 10,000,000
Maps, GPS Navigation & Directions, Street View : 10,000,000
MapQuest: Directions, Maps, GPS & Navigation : 10,000,000
DB Navigator : 10,000,000


In [66]:
print_android_genre('FAMILY',50)

Google Play Games : 1,000,000,000
Where's My Water? Free : 100,000,000
Tom Loves Angela : 100,000,000
Talking Tom Cat 2 : 100,000,000
Talking Tom Cat : 100,000,000
Talking Tom & Ben News : 100,000,000
Talking Ginger : 100,000,000
ROBLOX : 100,000,000
Minion Rush: Despicable Me Official Game : 100,000,000
Hay Day : 100,000,000
Farm Heroes Saga : 100,000,000
Duolingo: Learn Languages Free : 100,000,000
Bubble Witch 2 Saga : 100,000,000
Bitmoji – Your Personal Emoji : 100,000,000
aa : 50,000,000
YouTube Kids : 50,000,000
Voice changer with effects : 50,000,000
Vegas Crime Simulator : 50,000,000
Turbo FAST : 50,000,000
Township : 50,000,000
Tom's Love Letters : 50,000,000
Toca Kitchen 2 : 50,000,000
Talking Tom Bubble Shooter : 50,000,000
SpongeBob Diner Dash : 50,000,000
SimCity BuildIt : 50,000,000
Photo Warp : 50,000,000
Mobile Strike : 50,000,000
MY LITTLE PONY: Magic Princess : 50,000,000
MARVEL Future Fight : 50,000,000
Last Empire - War Z: Strategy : 50,000,000
LEGO® Juniors Create 

In [67]:
print_android_genre('SPORTS')

FIFA Soccer : 100,000,000
3D Bowling : 100,000,000
Wrestling Revolution 3D : 50,000,000
Top Eleven 2018 - Be a Soccer Manager : 50,000,000
NFL : 50,000,000
NBA LIVE Mobile Basketball : 50,000,000
EA SPORTS UFC® : 50,000,000
Cricbuzz - Live Cricket Scores & News : 50,000,000
3D Tennis : 50,000,000
theScore: Live Sports Scores, News, Stats & Videos : 10,000,000
beIN SPORTS TR : 10,000,000
WatchESPN : 10,000,000


In [68]:
print_android_genre('ART_AND_DESIGN')

Sketch - Draw & Paint : 50,000,000
ibis Paint X : 10,000,000
Textgram - write on photos : 10,000,000
Tattoo Name On My Photo Editor : 10,000,000
Canva: Poster, banner, card maker & graphic design : 10,000,000
U Launcher Lite – FREE Live Cool Themes, Hide Apps : 5,000,000
Floor Plan Creator : 5,000,000
FlipaClip - Cartoon animation : 5,000,000
Text on Photo - Fonteee : 1,000,000
Name Art Photo Editor - Focus n Filters : 1,000,000
Infinite Painter : 1,000,000
Garden Coloring Book : 1,000,000


In [69]:
print_android_genre('FOOD_AND_DRINK')

foodpanda - Local Food Delivery : 10,000,000
Zomato - Restaurant Finder and Food Delivery App : 10,000,000
Uber Eats: Local Food Delivery : 10,000,000
Tastely : 10,000,000
Starbucks : 10,000,000
Pizza Hut : 10,000,000
McDonald's - McDonald's Japan : 10,000,000
McDonald's : 10,000,000
Foursquare City Guide : 10,000,000
Domino's Pizza USA : 10,000,000
Delivery yogi. : 10,000,000
Cookpad - FREE recipe search makes fun cooking · musical making! : 10,000,000


In [70]:
print_android_genre('EDUCATION')

Remind: School Communication : 10,000,000
Quizlet: Learn Languages & Vocab with Flashcards : 10,000,000
Math Tricks : 10,000,000
Lumosity: #1 Brain Games & Cognitive Training App : 10,000,000
Learn languages, grammar & vocabulary with Memrise : 10,000,000
Learn English with Wlingua : 10,000,000
Google Classroom : 10,000,000
ClassDojo : 10,000,000
Rosetta Stone: Learn to Speak & Read New Languages : 5,000,000
NeuroNation - Focus and Brain Training : 5,000,000
Mermaids : 5,000,000
Learn English Words Free : 5,000,000


In [71]:
print_android_genre('BUSINESS')

OfficeSuite : Free Office + PDF Editor : 100,000,000
File Commander - File Manager/Explorer : 100,000,000
Vault-Hide SMS,Pics & Videos,App Lock,Cloud backup : 50,000,000
Secure Folder : 50,000,000
Indeed Job Search : 50,000,000
Facebook Pages Manager : 50,000,000
Docs To Go™ Free Office Suite : 50,000,000
ZOOM Cloud Meetings : 10,000,000
Visual Voicemail by MetroPCS : 10,000,000
Uber Driver : 10,000,000
Tiny Scanner - PDF Scanner App : 10,000,000
Skype for Business for Android : 10,000,000


In [72]:
print_android_genre('LIFESTYLE')

Tinder : 100,000,000
Samsung+ : 50,000,000
metroZONE : 10,000,000
iHoroscope - 2018 Daily Horoscope & Astrology : 10,000,000
happn – Local dating app : 10,000,000
Zara : 10,000,000
Timely Alarm Clock : 10,000,000
Talking Babsy Baby: Baby Games : 10,000,000
Sleep as Android: Sleep cycle tracker, smart alarm : 10,000,000
Rate Guide Bill Letter : 10,000,000
OK cashbag [point of pleasure] : 10,000,000
Muslim Pro - Prayer Times, Azan, Quran & Qibla : 10,000,000


In [73]:
print_android_genre('FINANCE')

Google Pay : 100,000,000
PayPal : 50,000,000
İşCep : 10,000,000
Wells Fargo Mobile : 10,000,000
Mobile Bancomer : 10,000,000
K PLUS : 10,000,000
Itau bank : 10,000,000
HDFC Bank MobileBanking : 10,000,000
Credit Karma : 10,000,000
Chase Mobile : 10,000,000
Cash App : 10,000,000
Capital One® Mobile : 10,000,000


In [74]:
print_android_genre('HOUSE_AND_HOME')

tinyCam Monitor FREE : 10,000,000
Zillow: Find Houses for Sale & Apartments for Rent : 10,000,000
Trulia Real Estate & Rentals : 10,000,000
Realtor.com Real Estate: Homes for Sale and Rent : 10,000,000
Houzz Interior Design Ideas : 10,000,000
Trulia Rent Apartments & Homes : 5,000,000
Real Estate sale & rent Trovit : 5,000,000
DaBang - Rental Homes in Korea : 5,000,000
Alfred Home Security Camera : 5,000,000
realestate.com.au - Buy, Rent & Sell Property : 1,000,000
Zumper - Apartment Rental Finder : 1,000,000
Wall Decorating Ideas : 1,000,000


In [75]:
print_android_genre('DATING')

Zoosk Dating App: Meet Singles : 10,000,000
OkCupid Dating : 10,000,000
Moco - Chat, Meet People : 10,000,000
Match™ Dating - Meet Singles : 10,000,000
Hot or Not - Find someone right now : 10,000,000
Hitwe - meet people and chat : 10,000,000
Find Real Love — YouLove Premium Dating : 10,000,000
Chat Rooms, Avatars, Date - Galaxy : 10,000,000
iPair-Meet, Chat, Dating : 5,000,000
eharmony - Online Dating App : 5,000,000
Free Dating App & Flirt Chat - Match with Singles : 5,000,000
Free Dating & Flirt Chat - Choice of Love : 5,000,000


In [76]:
print_android_genre('COMICS')

LINE WEBTOON - Free Comics : 10,000,000
comico Popular Original Cartoon Updated Everyday Comico : 5,000,000
Perfect Viewer : 5,000,000
Narrator's Voice : 5,000,000
Comics : 5,000,000
漫咖 Comics - Manga,Novel and Stories : 1,000,000
pixiv comic - everyone's manga app : 1,000,000
WebComics : 1,000,000
Tapas – Comics, Novels, and Stories : 1,000,000
Memes Button : 1,000,000
Marvel Unlimited : 1,000,000
Manga Zero - Japanese cartoon and comic reader : 1,000,000


In [77]:
print_android_genre('AUTO_AND_VEHICLES')

AutoScout24 - used car finder : 10,000,000
Android Auto - Maps, Media, Messaging & Voice : 10,000,000
Used cars for sale - Trovit : 5,000,000
Ulysse Speedometer : 5,000,000
Fines of the State Traffic Safety Inspectorate are official: inspection, payment of fines : 5,000,000
ezETC (ETC balance inquiry, meter trial, real-time traffic) : 1,000,000
auto fines : 1,000,000
Used car search Goo net whole car Go to net : 1,000,000
Used car is the first car - used car purchase, used car quotation, dealer information to : 1,000,000
Used Cars and Trucks for Sale : 1,000,000
Tickets + PDA 2018 Exam : 1,000,000
SKencar : 1,000,000


In [78]:
print_android_genre('LIBRARIES_AND_DEMO')

MX Player Codec (ARMv7) : 10,000,000
Cardboard : 10,000,000
Aviary Stickers: Free Pack : 10,000,000
Wifi Connect Library : 5,000,000
Aviary Effects: Classic : 5,000,000
Wifi Test : 1,000,000
OpenCV Manager : 1,000,000
New Ringtones 2018 : 1,000,000
Market Update Helper : 1,000,000
ES Chromecast plugin : 1,000,000
Cool Popular Ringtones 2018 🔥 : 1,000,000
Apk Installer : 1,000,000


In [79]:
print_android_genre('PARENTING')

Pregnancy Tracker & Countdown to Baby Due Date : 10,000,000
My baby Piano : 5,000,000
Vegetable Fun : 1,000,000
Urban Limo Taxi Simulator : 1,000,000
Safe365 – Cell Phone GPS Locator For Your Family : 1,000,000
My baby firework : 1,000,000
My baby Phone : 1,000,000
My baby Game (Balloon POP!) : 1,000,000
Feed Baby - Baby Tracker : 1,000,000
Family Album Mitene: Private Photo & Video Sharing : 1,000,000
Dormi - Baby Monitor : 1,000,000
Baby Sleep: White noise lullabies for newborns : 1,000,000


In [80]:
print_android_genre('BEAUTY')

Beauty Camera - Selfie Camera : 10,000,000
Best Hairstyles step by step : 5,000,000
ipsy: Makeup, Beauty, and Tips : 1,000,000
Ulta Beauty : 1,000,000
Sephora: Skin Care, Beauty Makeup & Fragrance Shop : 1,000,000
Selfie Camera : 1,000,000
Rainbow Camera : 1,000,000
Mirror Camera (Mirror + Selfie Camera) : 1,000,000
Mirror - Zoom & Exposure - : 1,000,000
Filters for Selfie : 1,000,000
Sticker Camera - Selfie Filters, Beauty Camera : 500,000
Hush - Beauty for Everyone : 500,000


In [81]:
print_android_genre('EVENTS')

Ticketmaster Event Tickets : 5,000,000
StubHub - Tickets to Sports, Concerts & Events : 5,000,000
Vivid Seats – Event Tickets : 1,000,000
SeatGeek – Tickets to Sports, Concerts, Broadway : 1,000,000
Gametime - Tickets to Sports, Concerts, Theater : 1,000,000
Fever : 1,000,000
Reminder : 500,000
vide-greniers.org : 100,000
my4D : 100,000
Xceed - Clubs, DJs, Festivals & Tickets : 100,000
Quake & Volcanoes: 3D Globe of Volcanic Eruptions : 100,000
Picktrainer: India's largest photo contest app : 100,000


In [82]:
print_android_genre('MEDICAL')

My Calendar - Period Tracker : 5,000,000
Blood Pressure : 5,000,000
mySugr: the blood sugar tracker made just for you : 1,000,000
Pregnancy Week By Week : 1,000,000
Pregnancy Calculator and Tracker app : 1,000,000
Ovia Pregnancy Tracker & Baby Countdown Calendar : 1,000,000
Ovia Fertility Tracker & Ovulation Calculator : 1,000,000
MyChart : 1,000,000
GoodRx Drug Prices and Coupons : 1,000,000
FollowMyHealth® : 1,000,000
Epocrates Plus : 1,000,000
Drugs.com Medication Guide : 1,000,000
